In [1]:
# !pip install -q neo4j wikipedia lxml gpt4all


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO_DB = os.getenv('NEO_DB')
NEO_USER = os.getenv('NEO_USER')
NEO_PASS = os.getenv('NEO_PASS')



In [3]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import GPT4AllEmbeddings

# Load documents

In [4]:


# Read the wikipedia article
raw_documents = WikipediaLoader(query="The Witcher").load()
# Define chunking strategy
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)
# Chunk the document
documents = text_splitter.split_documents(raw_documents)
# Remove the summary
for d in documents:
    del d.metadata["summary"]



/opt/conda/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/conda/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [5]:
embeddings = GPT4AllEmbeddings()

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [6]:
url="neo4j://192.168.68.84:7687/"

neo4j_db = Neo4jVector.from_documents(
    documents,
    embeddings,
    url=url,
    username=NEO_USER,
    password= NEO_PASS,
    #database=NEO_DB,  # neo4j by default
    index_name="wikipedia",  # vector by default
    node_label="WikipediaArticle",  # Chunk by default
    text_node_property="info",  # text by default
    embedding_node_property="vector",  # embedding by default
    create_id_index=True,  # True by default
)

In [7]:
existing_index = Neo4jVector.from_existing_index(
    embedding=embeddings,
    url=url,
    username=NEO_USER,
    password=NEO_PASS,
    index_name="wikipedia",
    text_node_property="info",  # Need to define if it is not default
)

In [8]:
print(existing_index.node_label) # WikipediaArticle
print(existing_index.embedding_node_property) # vector

WikipediaArticle
vector


In [9]:
query = "Who was the witcher in the video game?"
docs_with_score = existing_index.similarity_search_with_score(query, k=3)

In [10]:
len(docs_with_score)

3

In [11]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content[:200])
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.8134828805923462
The Witcher (Polish: Wiedźmin) is a fantasy action role-playing game series developed by CD Projekt Red and published by CD Projekt. It is based on the book series of the same name by Polish writer An
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.8134828805923462
The Witcher (Polish: Wiedźmin) is a fantasy action role-playing game series developed by CD Projekt Red and published by CD Projekt. It is based on the book series of the same name by Polish writer An
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.8134828805923462
The Witcher (Polish: Wiedźmin) is a fantasy action role-playing game series developed by CD Projekt Red and publ

# Information Extraction 

In [12]:
# GET DOCS

# CHUNK 

# LLM TO CLEAN 

# LLM TO FORMAT 

In [13]:
len(raw_documents)

23

In [14]:
#!pip -q install -U pip setuptools wheel
#!python -m -q spacy download en_core_web_sm
#!pip install jupyter_contrib_nbextensions
#!pip install --upgrade --no-deps --force-reinstall crosslingual_coreference 
#!pip install -U  allennlp nltk 'spacy[cuda12x,lookups]'


In [18]:

import spacy
import crosslingual_coreference

DEVICE = 0 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe("xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": DEVICE})

[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ModuleNotFoundError: No module named 'allennlp'